<a href="https://colab.research.google.com/github/mmaruthi/S4-Assignment-Solution/blob/master/S4-Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3, padding=1,bias=False)   # RF -> 3, inp - 28x28x1  out - 28x28x8
        self.conv1e = nn.Conv2d(8, 16, 3, padding=1,bias=False)   # RF -> 3, inp - 28x28x1  out - 28x28x16
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1,bias=False) # RF -> 5, inp - 28x28x16  out - 28x28x32
        self.conv1a = nn.Conv2d(32,16,1,bias=False)             # Using 1x1  to scale down the channels from 32 to 16
        self.bn1   = nn.BatchNorm2d(16)                         # Batch Normalization 
        self.pool  = nn.MaxPool2d(2, 2)                         #  RF -> 6, inp - 28x28x16 out - 14x14x16
        self.drop =  nn.Dropout2d(0.09)                          # Dropout 
        self.conv3 = nn.Conv2d(16, 16, 3, padding=1,bias=False) #  RF -> 10, ip - 14x14x16 o/p - 14x14x16
        self.conv4 = nn.Conv2d(16, 32, 3, padding=1,bias=False) # RF -> 14, ip 14x14x16 o/p - 14x14x32
        self.conv1b = nn.Conv2d(32,16,1,bias=False)             # Using 1x1  to scale down the channels from 32 to 16
        self.bn2   = nn.BatchNorm2d(16)                         # Batch Normalization
        self.pool  = nn.MaxPool2d(2, 2)                         #  RF -> 16, ip - 14x14x16 o/p - 7x7x16
        self.conv5 = nn.Conv2d(16, 16, 3, bias=False)           # RF -> 24, ip 7x7x16 o/p - 5x5x16
        self.conv6 = nn.Conv2d(16, 16, 3, bias=False)           # RF -> 40, ip 7x7x16 o/p - 5x5x16
        self.gap   = nn.AdaptiveAvgPool2d((1,1))                #Average Pooling
        self.fc    = nn.Linear(16,10,bias=False)                #FC Layer 

    def forward(self, x):
        x = F.relu(self.conv1e(F.relu(self.conv1(x))))
        x = F.relu(self.conv2(x))
        x = self.drop(self.pool(self.bn1(self.conv1a(x))))
        x = F.relu(self.conv4(F.relu(self.conv3(x))))
        x = self.drop(self.pool(self.bn2(self.conv1b(x))))
        x = F.relu(self.conv5(x))
        x = self.gap(x)
        x = x.view(-1, 16)
        x = self.fc(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)
 

In [74]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              72
            Conv2d-2           [-1, 16, 28, 28]           1,152
            Conv2d-3           [-1, 32, 28, 28]           4,608
            Conv2d-4           [-1, 16, 28, 28]             512
       BatchNorm2d-5           [-1, 16, 28, 28]              32
         MaxPool2d-6           [-1, 16, 14, 14]               0
         Dropout2d-7           [-1, 16, 14, 14]               0
            Conv2d-8           [-1, 16, 14, 14]           2,304
            Conv2d-9           [-1, 32, 14, 14]           4,608
           Conv2d-10           [-1, 16, 14, 14]             512
      BatchNorm2d-11           [-1, 16, 14, 14]              32
        MaxPool2d-12             [-1, 16, 7, 7]               0
        Dropout2d-13             [-1, 16, 7, 7]               0
           Conv2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=False, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [77]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.03, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0844, Accuracy: 9737/10000 (97%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0670, Accuracy: 9792/10000 (98%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0228, Accuracy: 9924/10000 (99%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0224, Accuracy: 9930/10000 (99%)



loss=0.01847418211400509 batch_id=468: 100%|██████████| 469/469 [00:11<00:00, 41.08it/s]



Test set: Average loss: 0.0207, Accuracy: 9934/10000 (99%)

